In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import folium
from folium.plugins import MarkerCluster
import h3

In [31]:
df = pd.read_csv('../../../data/processed/land_dataset_correct_addr.csv')

In [32]:
df.drop(columns=['latitude'], inplace=True)

In [33]:
df.head()

,price,land_area,longitude,price_per_m2,geometry,index_right,population,h3,ADM3_EN,ADM2_EN,ADM1_EN,polygon_geom
0,420000.0,420.0,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Tumpung Ti Muoy,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
1,420000.0,420.0,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Tumpung Ti Pir,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
2,420000.0,420.0,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Boeng Trabaek,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
3,420000.0,420.0,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Boeng Keng Kang Ti Bei,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
4,420000.0,420.0,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Svay Prey Ti Muoy,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...


In [34]:

def detect_hexagon_outliers(df, hex_col='h3', price_col='price_per_m2'):
    """
    Detect price outliers within each H3 hexagon using IQR method
    
    Args:
        df (pd.DataFrame): Input DataFrame with property data
        hex_col (str): Column name for H3 hexagon IDs
        price_col (str): Column name for price per m²
        
    Returns:
        tuple: (DataFrame with outlier flags, hexagon report)
    """
    # Create output column
    df['hex_outlier'] = False
    df['hex_outlier_type'] = np.nan  # 'high', 'low', or NaN for non-outliers
    
    # Prepare to store results
    outlier_counts = []
    hexagons = df[hex_col].unique()
    
    for hex_id in hexagons:
        hex_data = df[df[hex_col] == hex_id]
        prices = hex_data[price_col]
        
        # Skip hexagons with too few properties
        if len(prices) < 5:
            continue
            
        # Calculate IQR-based outlier bounds
        q1 = prices.quantile(0.25)
        q3 = prices.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # Identify outliers
        low_outliers = (prices < lower_bound)
        high_outliers = (prices > upper_bound)
        
        # Update DataFrame
        df.loc[low_outliers.index, 'hex_outlier'] = True
        df.loc[high_outliers.index, 'hex_outlier'] = True
        df.loc[low_outliers.index, 'hex_outlier_type'] = 'low'
        df.loc[high_outliers.index, 'hex_outlier_type'] = 'high'
        
        # Store results for reporting
        outlier_counts.append({
            'h3': hex_id,
            'n_properties': len(prices),
            'n_low_outliers': low_outliers.sum(),
            'n_high_outliers': high_outliers.sum(),
            'outlier_rate': (low_outliers.sum() + high_outliers.sum()) / len(prices),
            'median_price': prices.median(),
            'q1_price': q1,
            'q3_price': q3,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'commune': hex_data['ADM3_EN'].mode()[0] if not hex_data['ADM3_EN'].empty else 'Unknown',
            'district': hex_data['ADM2_EN'].mode()[0] if not hex_data['ADM2_EN'].empty else 'Unknown'
        })
    
    # Create report
    report_df = pd.DataFrame(outlier_counts)
    
    return df, report_df

def visualize_outliers(df, report_df):
    """Generate visualizations of hexagon outliers"""
    plt.figure(figsize=(18, 15))
    
    # 1. Hexagon outlier distribution
    plt.subplot(2, 2, 1)
    sns.histplot(report_df['outlier_rate'], bins=20, kde=True)
    plt.title('Distribution of Outlier Rates per Hexagon')
    plt.xlabel('Proportion of Outliers')
    plt.grid(True)
    
     # 2. Outliers vs hexagon size
    plt.subplot(2, 2, 2)
    sns.scatterplot(
        x='n_properties', 
        y='outlier_rate', 
        data=report_df,
        size='median_price',
        hue='median_price',
        palette='viridis',
        alpha=0.7
    )
    plt.title('Outlier Rate vs Properties per Hexagon')
    plt.xlabel('Number of Properties')
    plt.ylabel('Outlier Rate')
    plt.grid(True)
    
    # 3. Price distribution in hexagons with outliers
    plt.subplot(2, 2, 3)
    hex_with_outliers = report_df[report_df['n_low_outliers'] + report_df['n_high_outliers'] > 0]['h3']
    subset = df[df['h3'].isin(hex_with_outliers)]
    
    # Plot each hexagon's price distribution
    for hex_id in hex_with_outliers[:10]:  # Limit to first 10 for clarity
        hex_data = subset[subset['h3'] == hex_id]
        sns.kdeplot(hex_data['price_per_m2'], label=f'Hex {hex_id[:8]}...', fill=True, alpha=0.3)
    
    plt.title('Price Distributions in Hexagons with Outliers')
    plt.xlabel('Price per m²')
    plt.ylabel('Density')
    plt.legend()
    
    
    # 4. Map view of outlier locations
    plt.subplot(2, 2, 4)
    outlier_colors = {'high': 'red', 'low': 'green', np.nan: 'blue'}
    colors = df['hex_outlier_type'].map(outlier_colors)
    
    plt.scatter(
        df['longitude'], 
        df['latitude'], 
        c=colors,
        s=df['price_per_m2']/100,  # Marker size by price
        alpha=0.5
    )
    plt.title('Geographic Distribution of Outliers')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    
    
    # 4. Map view of outlier locations
    plt.subplot(2, 2, 4)
    outlier_colors = {'high': 'red', 'low': 'green', np.nan: 'blue'}
    colors = df['hex_outlier_type'].map(outlier_colors)
    
    plt.scatter(
        df['longitude'], 
        df['latitude'], 
        c=colors,
        s=df['price_per_m2']/100,  # Marker size by price
        alpha=0.5
    )
    plt.title('Geographic Distribution of Outliers')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    
    # Create custom legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='High Outlier'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Low Outlier'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Normal')
    ]
    plt.legend(handles=legend_elements)
    
    plt.tight_layout()
    plt.savefig('hexagon_outliers_analysis.png', dpi=300)
    plt.show()

In [35]:
def create_interactive_map(df):
    """Create interactive Folium map showing outliers"""
    # Center map on Phnom Penh
    map_center = [df['latitude'].mean(), df['longitude'].mean()]
    m = folium.Map(location=map_center, zoom_start=13, tiles='CartoDB positron')
    
    # Create marker clusters
    normal_cluster = MarkerCluster(name='Normal Prices').add_to(m)
    low_cluster = MarkerCluster(name='Low Outliers').add_to(m)
    high_cluster = MarkerCluster(name='High Outliers').add_to(m)
    
    # Add markers
    for _, row in df.iterrows():
        popup_content = f"""
        <b>Price/m²:</b> ${row['price_per_m2']:,.0f}<br>
        <b>Total Price:</b> ${row['price']:,.0f}<br>
        <b>Land Area:</b> {row['land_area']} m²<br>
        <b>Commune:</b> {row['ADM3_EN']}<br>
        <b>District:</b> {row['ADM2_EN']}<br>
        <b>Hex ID:</b> {row['h3']}
        """
        
        if row['hex_outlier_type'] == 'low':
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=5,
                color='green',
                fill=True,
                fill_color='green',
                fill_opacity=0.7,
                popup=popup_content
            ).add_to(low_cluster)
            
        elif row['hex_outlier_type'] == 'high':
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=5 + row['price_per_m2']/1000,  # Scale by price
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=0.7,
                popup=popup_content
            ).add_to(high_cluster)
            
        else:
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=3,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.3,
                popup=popup_content
            ).add_to(normal_cluster)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Save map
    m.save('hexagon_outliers_map.html')
    print("Interactive map saved to hexagon_outliers_map.html")

In [36]:
def generate_hexagon_report(report_df):
    """Generate comprehensive report for top hexagons with outliers"""
    print("\n===== Hexagon Outlier Analysis Report =====")
    print(f"Total hexagons analyzed: {len(report_df)}")
    print(f"Hexagons with outliers: {len(report_df[report_df['n_low_outliers'] + report_df['n_high_outliers'] > 0])}")
    
    # Top hexagons by outlier count
    report_df['total_outliers'] = report_df['n_low_outliers'] + report_df['n_high_outliers']
    top_outliers = report_df.sort_values('total_outliers', ascending=False).head(10)
    
    print("\nTop 10 Hexagons by Outlier Count:")
    print(top_outliers[['h3', 'commune', 'district', 'n_properties', 'total_outliers', 
                        'n_low_outliers', 'n_high_outliers', 'median_price']])
    
    # Most expensive hexagons
    top_expensive = report_df.sort_values('median_price', ascending=False).head(5)
    print("\nTop 5 Most Expensive Hexagons:")
    print(top_expensive[['h3', 'commune', 'district', 'median_price', 'n_properties']])
    
    # Hexagons with highest outlier rates
    high_rate = report_df[report_df['n_properties'] > 10].sort_values('outlier_rate', ascending=False).head(5)
    print("\nHexagons with Highest Outlier Rates (>10 properties):")
    print(high_rate[['h3', 'commune', 'district', 'outlier_rate', 'n_properties', 'median_price']])


In [37]:
# # Ensure we have required columns
# required_cols = ['h3', 'price_per_m2', 'longitude', 'latitude', 'ADM3_EN', 'ADM2_EN']
# missing = [col for col in required_cols if col not in df.columns]
# if missing:
#     print(f"Missing columns: {missing}")
# else:
#     # Detect outliers
#     df_with_outliers, report = detect_hexagon_outliers(df)
    
#     # Visualize results
#     visualize_outliers(df_with_outliers, report)
#     create_interactive_map(df_with_outliers)
# # Generate report
# generate_hexagon_report(report)

# # Save results
# df_with_outliers.to_csv('property_data_with_outliers.csv', index=False)
# report.to_csv('hexagon_outlier_report.csv', index=False)

# print(f"\nIdentified {df_with_outliers['hex_outlier'].sum()} hexagon-based outliers")
# print(f" - High outliers: {len(df_with_outliers[df_with_outliers['hex_outlier_type'] == 'high'])}")
# print(f" - Low outliers: {len(df_with_outliers[df_with_outliers['hex_outlier_type'] == 'low'])}")

In [38]:
def remove_hexagon_outliers(df, hex_col='h3', price_col='price_per_m2'):
    """
    Remove price outliers within each H3 hexagon using IQR method
    
    Args:
        df (pd.DataFrame): Input DataFrame with property data
        hex_col (str): Column name for H3 hexagon IDs
        price_col (str): Column name for price per m²
        
    Returns:
        pd.DataFrame: DataFrame with outliers removed
    """
    # Create a copy to avoid modifying original data
    cleaned_df = df.copy()
    
    # List to store indices of outliers
    outlier_indices = []
    
    for hex_id in cleaned_df[hex_col].unique():
        hex_data = cleaned_df[cleaned_df[hex_col] == hex_id]
        
        # Skip hexagons with too few properties
        if len(hex_data) < 5:
            continue
            
        # Calculate IQR-based outlier bounds
        prices = hex_data[price_col]
        q1 = prices.quantile(0.25)
        q3 = prices.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # Identify outliers
        hex_outliers = hex_data[(prices < lower_bound) | (prices > upper_bound)]
        outlier_indices.extend(hex_outliers.index.tolist())
    
    # Remove outliers
    cleaned_df = cleaned_df.drop(outlier_indices)
    
    # Report statistics
    n_removed = len(outlier_indices)
    pct_removed = n_removed / len(df) * 100
    print(f"Removed {n_removed} outliers ({pct_removed:.2f}% of dataset)")
    print(f"Final dataset size: {len(cleaned_df)} properties")
    
    return cleaned_df

In [39]:
# # Remove outliers
# cleaned_df = remove_hexagon_outliers(df)

# # Save cleaned data
# cleaned_df.to_csv('property_data_no_outliers.csv', index=False)

# # Visualize before/after comparison
# plt.figure(figsize=(15, 6))

# # Before cleaning
# plt.subplot(1, 2, 1)
# plt.scatter(df['longitude'], df['latitude'], 
#             c=df['price_per_m2'], cmap='viridis', 
#             s=10, alpha=0.7)
# plt.colorbar(label='Price per m²')
# plt.title('Original Data with Outliers')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')

# # After cleaning
# plt.subplot(1, 2, 2)
# plt.scatter(cleaned_df['longitude'], cleaned_df['latitude'], 
#             c=cleaned_df['price_per_m2'], cmap='viridis', 
#             s=10, alpha=0.7)
# plt.colorbar(label='Price per m²')
# plt.title('Cleaned Data Without Outliers')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')

# plt.tight_layout()
# plt.savefig('outlier_removal_comparison.png', dpi=300)
# plt.show()

In [40]:
cleaned_df

,price,land_area,longitude,price_per_m2,geometry,index_right,population,h3,ADM3_EN,ADM2_EN,ADM1_EN,polygon_geom
0,420000.0,420.0,104.913586,1000.000000,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Tumpung Ti Muoy,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
1,420000.0,420.0,104.913586,1000.000000,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Tumpung Ti Pir,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
2,420000.0,420.0,104.913586,1000.000000,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Boeng Trabaek,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
3,420000.0,420.0,104.913586,1000.000000,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Boeng Keng Kang Ti Bei,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
4,420000.0,420.0,104.913586,1000.000000,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Svay Prey Ti Muoy,Chamkar Mon,Phnom Penh,MULTIPOLYGON (((104.92182899858238 11.54190394...
...,...,...,...,...,...,...,...,...,...,...,...,...
10287,50000.0,480.0,104.920096,104.166667,POINT (104.920096 11.575837),53920.0,16252.0,8865846aadfffff,Monourom,Prampir Meakkakra,Phnom Penh,MULTIPOLYGON (((104.92501668529559 11.57292497...
10288,50000.0,480.0,104.920096,104.166667,POINT (104.920096 11.575837),53920.0,16252.0,8865846aadfffff,Srah Chak,Doun Penh,Phnom Penh,MULTIPOLYGON (((104.92501668529559 11.57292497...
10289,50000.0,480.0,104.920096,104.166667,POINT (104.920096 11.575837),53920.0,16252.0,8865846aadfffff,Voat Phnum,Doun Penh,Phnom Penh,MULTIPOLYGON (((104.92501668529559 11.57292497...
10290,796000.0,7960.0,104.849064,100.000000,POINT (104.849064 11.43934032),54745.0,751.0,88658460c5fffff,Kong Noy,Dangkao,Phnom Penh,MULTIPOLYGON (((104.85642343546652 11.43898307...


In [41]:
import plotly.express as px
import plotly.io as pio

figs = []
for addr in cleaned_df['h3'].unique():
    subset = cleaned_df[cleaned_df['h3'] == addr]
    fig = px.box(subset, x='price_per_m2', points='all', title=f'Price per m²: {addr}')
    figs.append(fig)

with open("all_price_per_m2_grouped_plots.html", "w") as f:
    for fig in figs:
        f.write(pio.to_html(fig, full_html=False, include_plotlyjs='cdn'))

In [42]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# def detect_hexagon_outliers(df, hex_col='h3', price_col='price_per_m2', show_plots=True):
#     """
#     Detect price outliers within each H3 hexagon using IQR method
    
#     Args:
#         df (pd.DataFrame): Input DataFrame with property data
#         hex_col (str): Column name for H3 hexagon IDs
#         price_col (str): Column name for price per m²
#         show_plots (bool): Whether to display outlier visualizations
        
#     Returns:
#         pd.DataFrame: Original DataFrame with new 'hex_outlier' column
#     """
#     # Create output column
#     df['hex_outlier'] = False
    
#     # Prepare to store results
#     outlier_counts = []
#     hexagons = df[hex_col].unique()
    
#     for hex_id in hexagons:
#         hex_data = df[df[hex_col] == hex_id]
#         prices = hex_data[price_col]
        
#         # Skip hexagons with too few properties
#         if len(prices) < 5:
#             df.loc[df[hex_col] == hex_id, 'hex_outlier'] = False
#             continue
            
#         # Calculate IQR-based outlier bounds
#         q1 = prices.quantile(0.25)
#         q3 = prices.quantile(0.75)
#         iqr = q3 - q1
#         lower_bound = q1 - 1.5 * iqr
#         upper_bound = q3 + 1.5 * iqr
        
#         # Identify outliers
#         outliers = (prices < lower_bound) | (prices > upper_bound)
#         df.loc[outliers.index, 'hex_outlier'] = outliers
        
#         # Store results for reporting
#         outlier_counts.append({
#             'h3': hex_id,
#             'n_properties': len(prices),
#             'n_outliers': outliers.sum(),
#             'outlier_rate': outliers.mean(),
#             'median_price': prices.median(),
#             'q1_price': q1,
#             'q3_price': q3
#         })
    
#     # Create report
#     report_df = pd.DataFrame(outlier_counts)
    
#     # Visualize results if requested
#     if show_plots:
#         visualize_outliers(df, report_df, hex_col, price_col)
    
#     return df, report_df

# def visualize_outliers(df, report_df, hex_col, price_col):
#     """Generate visualizations of hexagon outliers"""
#     plt.figure(figsize=(15, 10))
    
#     # 1. Hexagon outlier distribution
#     plt.subplot(2, 2, 1)
#     sns.histplot(report_df['outlier_rate'], bins=20, kde=True)
#     plt.title('Distribution of Outlier Rates per Hexagon')
#     plt.xlabel('Proportion of Outliers')
#     plt.grid(True)
    
#     # 2. Outliers vs hexagon size
#     plt.subplot(2, 2, 2)
#     sns.scatterplot(
#         x='n_properties', 
#         y='outlier_rate', 
#         data=report_df,
#         size='median_price',
#         hue='median_price',
#         palette='viridis',
#         alpha=0.7
#     )
#     plt.title('Outlier Rate vs Properties per Hexagon')
#     plt.xlabel('Number of Properties')
#     plt.ylabel('Outlier Rate')
#     plt.grid(True)
    
#     # 3. Price distribution in hexagons with outliers
#     plt.subplot(2, 2, 3)
#     hex_with_outliers = report_df[report_df['n_outliers'] > 0]['h3']
#     subset = df[df[hex_col].isin(hex_with_outliers)]
    
#     # Plot each hexagon's price distribution
#     for hex_id in hex_with_outliers[:10]:  # Limit to first 10 for clarity
#         hex_data = subset[subset[hex_col] == hex_id]
#         sns.kdeplot(hex_data[price_col], label=f'Hex {hex_id[:8]}...', fill=True, alpha=0.3)
    
#     plt.title('Price Distributions in Hexagons with Outliers')
#     plt.xlabel('Price per m²')
#     plt.ylabel('Density')
#     plt.legend()
    
#     # 4. Map view of outlier locations
#     plt.subplot(2, 2, 4)
#     plt.scatter(
#         df['longitude'], 
#         df['latitude'], 
#         c=df['hex_outlier'].map({True: 'red', False: 'blue'}),
#         s=df[price_col]/100,  # Marker size by price
#         alpha=0.5
#     )
#     plt.title('Geographic Distribution of Outliers')
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.colorbar(label='Outlier Status (Red=Outlier)')
    
#     plt.tight_layout()
#     plt.savefig('hexagon_outliers_analysis.png', dpi=300)
#     plt.show()

# # Example usage
# if __name__ == "__main__":
#     # Load your dataset
#     df = df.copy()  # Ensure we don't modify the original DataFrame
    
#     # Detect outliers
#     df_with_outliers, report = detect_hexagon_outliers(df)
    
#     # Save results
#     df_with_outliers.to_csv('property_data_with_outliers.csv', index=False)
#     report.to_csv('hexagon_outlier_report.csv', index=False)
    
#     print(f"Identified {df_with_outliers['hex_outlier'].sum()} hexagon-based outliers")
#     print("Top hexagons by outlier count:")
#     print(report.sort_values('n_outliers', ascending=False).head(10))